In [1]:
import numpy as np
import cv2
import json
import os
import argparse
import sys

In [2]:
example = True
example= example and example
example

True

In [3]:
True and (True and False)

False

In [107]:
class BoundingBox:
    '''
    Bounding box with corresponding parametes
    Params:
        center_x (float): x coordinate of the center
        center_y (float): y coordinate of the center
        width (float): width of the bounding box
        height (float): height of the bounding box
        label (str): label name
    '''
    def __init__(self, center_x: float, center_y: float, width: float, height: float, label: str):
        self.center_x = center_x
        self.center_y = center_y
        self.width = width
        self.height = height
        self.label = label
        
    def eq_bbs(self, other, threshold = 200):
        coords_condition = True
        coords_condition = coords_condition and abs(self.center_x - other.center_x)<threshold
        coords_condition = coords_condition and abs(self.center_y - other.center_y)<threshold
        coords_condition = coords_condition and abs(self.width - other.width)<threshold
        coords_condition = coords_condition and abs(self.height - other.height)<threshold
        return coords_condition
    def eq_labels(self, other):
        return self.label == other.label
    
    def __eq__(self, other, threshold = 200):
        '''
        Equality method
        Args:
            self: object to compare
            other: object to compare
            threshhold (int, can be float): threshhold for float BoundingBox's params estimation
        '''
#         label_condition = self.label == other.label     
#         coords_condition = True
#         coords_condition = coords_condition and abs(self.center_x - other.center_x)<threshold
#         coords_condition = coords_condition and abs(self.center_y - other.center_y)<threshold
#         coords_condition = coords_condition and abs(self.width - other.width)<threshold
#         coords_condition = coords_condition and abs(self.height - other.height)<threshold
        
        label_condition = self.eq_labels(other)
        coords_condition = self.eq_bbs(other, threshold=threshold)
        return label_condition and coords_condition
    
    def __hash__(self):
#         TODO: rewrite hash for set, because for set it doesn't work. Instead eq function is used to compare sets
        return hash((self.center_x, self.center_y, self.width, self.width, self.label))
    
    def __str__(self):
        return f'BoundingBox({self.label}, [{self.center_x}, {self.center_y}], [{self.width}, {self.height}])'
    def __repr__(self):
        return f'BoundingBox({self.label}, [{self.center_x}, {self.center_y}], [{self.width}, {self.height}])'

In [135]:
# def eq(bbs1, bbs2):
#     '''
#     Compares two sets of BoundingBox objects (dumb method)
#     '''
#     eq_result = [True, True]
#     result = 0
#     for bb1 in bbs1:
#         for bb2 in bbs2:
#             if bb1==bb2:
#                 result+=1
#                 break
#     print(result, len(bbs1))
#     if result == min(len(bbs1), len(bbs2)):
#         eq_result[0] = True
#         if 
#     return result == len(bbs1) and len(bbs1) == len(bbs2)



def eq(bb_set1, bb_set2):
    '''
    Compares two sets of BoundingBox objects (dumb method)
    '''
#     eq_result = [False, False]
    eq_result = [False, False, False, False, False]
    result = 0
    bbs_result = 0
    bbs1 = bb_set1.copy()
    bbs2 = bb_set2.copy()
    
    if len(bbs1)>len(bbs2):
        bbs1, bbs2 = bbs2, bbs1
    bbs2_length = len(bbs2)
    
    
    
    for bb1 in bbs1:
        for bb2 in bbs2:
            if bb1.eq_bbs(bb2):
                bbs_result+=1
                bbs2.remove(bb2)
                break
    
    bbs1 = bb_set1.copy()
    bbs2 = bb_set2.copy()
    bbs2_length = len(bbs2)
    if len(bbs1)>len(bbs2):
        bbs1, bbs2 = bbs2, bbs1
    
    bbs2_length = len(bbs2)
    
    for bb1 in bbs1:
        for bb2 in bbs2:
            if bb1==bb2:
                result+=1
                bbs2.remove(bb2)
                break
    
    print('BBs results:')
    print(bbs_result, len(bbs1))
    print(bbs_result, bbs2_length)
    print('Results with labels:')
    print(result, len(bbs1))
    print(result, bbs2_length)
    
    if bbs_result>0:
        eq_result[0] = True
        if bbs_result>=len(bbs1)//2:
            eq_result[1] = True
            if bbs_result==len(bbs1):
                eq_result[2] = True
                if result == len(bbs1):
                    eq_result[3] = True
                    if len(bbs1) == bbs2_length:
                        eq_result[4]=True
            
    return eq_result

In [136]:
# def class_colors(names):
#     """
#     Create a dict with one random BGR color for each
#     class name
#     """
#     return {name: (
#         np.random.randint(0, 255),
#         np.random.randint(0, 255),
#         np.random.randint(0, 255)) for name in names}

# def compare_labels(json_path1, json_path2, colors = None, classes = None):
#     '''
#     Compare labels from json files
#     Args:
#         json_path1 (str): path to json file with labels
#         json_path2 (str): path to json file with labels
#         colors: list of RGB color for each bounding boxes class
#     Returns:
#         Comparison report
#     '''

#     # 1. Initialize variables with bounding boxes data
#     with open(json_path1) as f:
#         img_data1 = json.load(f)
#     with open(json_path2) as f:
#         img_data2 = json.load(f)
        
#     if colors is None:
#         colors = class_colors(classes)
    

#     # 2. Create two black images with corresponding bounding boxes
#     width, height = 2592, 1520 
#     img1 = 255 * np.ones((width, height, 3), dtype = np.uint8)
#     img2 = 255 * np.ones((width, height, 3), dtype = np.uint8)
#     img_list = [(img1, img_data1), (img2, img_data2)]
#     # loop in which  labels are added into corresponding image
#     for img, img_data in img_list:
#         shape_list = img_data['shapes']
#         for rect in shape_list:
#             label = rect['label']
#             left, top = list(map(lambda x: int(round(x)), (rect['points'][0])))
#             right, bottom = list(map(lambda x: int(round(x)), (rect['points'][1])))
#             cv2.rectangle(img, (left, top), (right, bottom), colors[label])
    
#     mse = ((img1-img2)**2).mean()
#     return mse


def get_xywh(rect):
    '''
    Gets x, y, width and height from x1, y1, x2, y2 coordinates
    '''
    x_list=[rect['points'][0][0], rect['points'][1][0]]
    y_list=[rect['points'][0][1], rect['points'][1][1]]
    x_max = max(x_list)
    x_min = min(x_list)
    y_max = max(y_list)
    y_min = min(y_list)

    width = x_max-x_min
    height = y_max-y_min
    center_x,center_y = x_min+width/2.0 , y_min+height/2.0

    return center_x, center_y, width, height

def compare_labels(json_path1, json_path2):
    '''
    Compares labels of two labeled images
    '''
    # 1. initialize variables with bounding boxes data
    with open(json_path1) as f:
        img_data1 = json.load(f)
    with open(json_path2) as f:
        img_data2 = json.load(f)
    
    # 2. Build two sets of BoundingBox objects    
    img_data_list = [img_data1, img_data2]
    bbs_list = []
    for img_data in img_data_list:
        shape_list = img_data['shapes']
#         bbs = {BoundingBox(*[coord for coords in rect['points'] for coord in coords], rect['label']) for rect in shape_list}
        bbs = {BoundingBox(*get_xywh(rect), rect['label']) for rect in shape_list}
        bbs_list.append(bbs)
    
    bbs1 = bbs_list[0]
    bbs2 = bbs_list[1]
#     print(bbs1, len(bbs1))
#     print(bbs2, len(bbs2))
    # 3. Compare two sets using eq function
    return eq(bbs1, bbs2)

In [137]:
# json_path1 = r'D:/Projects/metal_classification/data/examples/00000042(1).json'
# json_path2 = r'D:/Projects/metal_classification/data/examples/00000042(3).json'
json_path1 = r'D:/Projects/metal_classification/data/frames/user_frames/george/00000329.json'
json_path2 = r'D:/Projects/metal_classification/data/frames/user_frames/max/00000329.json'

compare_labels(json_path1, json_path2)

BBs results:
19 26
19 26
Results with labels:
7 26
7 26


[True, True, False, False, False]

In [138]:
import glob
json_path1 = r'D:/Projects/metal_classification/data/frames/user_frames/george'
json_path2 = r'D:/Projects/metal_classification/data/frames/user_frames/victor'
json_path1_list = glob.glob(os.path.join(json_path1, '*.json'))
json_path2_list = glob.glob(os.path.join(json_path2, '*.json'))

for json_path1, json_path2 in zip(json_path1_list, json_path2_list):
    print(os.path.basename(json_path1), os.path.basename(json_path2))
    print(compare_labels(json_path1, json_path2))
    print()

00000010.json 00000010.json
BBs results:
12 12
12 29
Results with labels:
11 12
11 29
[True, True, True, False, False]

00000038.json 00000038.json
BBs results:
11 12
11 22
Results with labels:
10 12
10 22
[True, True, False, False, False]

00000329.json 00000329.json
BBs results:
21 26
21 27
Results with labels:
14 26
14 27
[True, True, False, False, False]

00000636.json 00000636.json
BBs results:
15 15
15 28
Results with labels:
11 15
11 28
[True, True, True, False, False]

00000779.json 00000779.json
BBs results:
16 17
16 36
Results with labels:
12 17
12 36
[True, True, False, False, False]

00000834.json 00000834.json
BBs results:
7 7
7 9
Results with labels:
7 7
7 9
[True, True, True, True, False]

00000985.json 00000985.json
BBs results:
6 9
6 16
Results with labels:
3 9
3 16
[True, True, False, False, False]

00001146.json 00001146.json
BBs results:
21 22
21 33
Results with labels:
9 22
9 33
[True, True, False, False, False]

00001478.json 00001478.json
BBs results:
15 20
15 31